<a href="https://colab.research.google.com/github/anilaksu/Algorithmic-Trading-Codes/blob/main/Udemy_Triangular_Arbitrage_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Triangular Arbitrage for Crypto with Python Udemy Training**


Anil Aksu

Personal e-mail: aaa293@cornell.edu



**Outline:**


1.  CEFI Triangular Arbitrage
  * Synchronous vs Asynchronous
  * Blocking & Timeouts
  * Scraping with Selenium
  * Asynchronous Functions
  * Asynchronous Iterators
2. DEFI Triangular Arbitrage
  * Coroutine Objects and Async Functions
  * Ways of running coroutines
  * Cancelling coroutines
  * Awaitable Objects



Clone GitHub Repo:

git clone https://github.com/CryptoWizardsNet/poloniex-tri-arb



If you have never used GitHub before or do not have GitHub working on your command line:

https://www.youtube.com/watch?v=CKcqniGu3tA

https://git-scm.com/book/en/v2/Getting-Started-Installing-Git



Google Colab Notebook:

https://colab.research.google.com/drive/1gvX613_dwF5p6Hea9xMYeC-KokE9e9C_?usp=sharing



Poloniex latest docs:

https://docs.poloniex.com/#introduction



Binance latest docs:

https://binance-docs.github.io/apidocs/spot/en/#kline-candlestick-data

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/ColabNotebooks/FinanceAlgorithms
!ls # special shell command to view the files in the home directory of the notebook environment

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/ColabNotebooks/FinanceAlgorithms
 2013-03-08options.csv	      local.csv
 2013-03-08stocks.csv	      OptionsTrading.ipynb
'Asynchronous Python.ipynb'   PriceJump.gdraw
 async_scrape.py	     'Stock Markets Codes.ipynb'
 BinanceTradingPairs.xlsx     structured_triangular_pairs.json
 EURUSD_Options_Data.csv      triangular_pairs_bid_ask_prices.json
 EURUSD_Options_Data.gsheet  'Udemy Triangular Arbitrage Training.ipynb'
'ForEx&IndexData.xls'


#**1. How to pull any API Data**

In [ ]:
# Here we install required libraries for API data manipulation
!python3 -V
!which pip3
!pip3 install requests --upgrade --no-cache-dir

Python 3.10.12
/usr/local/bin/pip3


In [ ]:
# Here we import required libraries
import requests
import json

In [ ]:
# Poloniex prices
prices = requests.get("https://api.poloniex.com/markets/price")
prices_list = []

if prices.status_code == 200:
  prices_json = json.loads(prices.text)
  for p in prices_json:
    prices_list.append({"ticker": p["symbol"], "price" : p["price"]})

In [ ]:
print(prices_list)

[{'ticker': 'BTS_BTC', 'price': '0.0000000625'}, {'ticker': 'DASH_BTC', 'price': '0.000624'}, {'ticker': 'DOGE_BTC', 'price': '0.0000025'}, {'ticker': 'LTC_BTC', 'price': '0.001401'}, {'ticker': 'XLM_BTC', 'price': '0.00000224'}, {'ticker': 'XEM_BTC', 'price': '0.000000515'}, {'ticker': 'XMR_BTC', 'price': '0.002226'}, {'ticker': 'XRP_BTC', 'price': '0.00000983'}, {'ticker': 'BTC_USDT', 'price': '64959.13'}, {'ticker': 'DASH_USDT', 'price': '39.5'}, {'ticker': 'LTC_USDT', 'price': '91.135'}, {'ticker': 'XLM_USDT', 'price': '0.1454'}, {'ticker': 'XMR_USDT', 'price': '144.98'}, {'ticker': 'XRP_USDT', 'price': '0.6387'}, {'ticker': 'ETH_BTC', 'price': '0.05428'}, {'ticker': 'ETH_USDT', 'price': '3524.25'}, {'ticker': 'SC_BTC', 'price': '0.00000014'}, {'ticker': 'DCR_BTC', 'price': '0.000402'}, {'ticker': 'LSK_BTC', 'price': '0.0000185'}, {'ticker': 'STEEM_BTC', 'price': '0.00000459'}, {'ticker': 'ETC_BTC', 'price': '0.000528'}, {'ticker': 'ETC_ETH', 'price': '0.00965'}, {'ticker': 'ETC_US

In [ ]:
# Poloniex Orderbook
ticker = "DASH_BTC"
limit = 20
# Structure Orderbook
ask_prices = []
ask_sizes = []
counts = 0

ob = requests.get(f"https://api.poloniex.com/markets/{ticker}/orderBook?limit={limit}")

if ob.status_code == 200:
  ob_json = json.loads(ob.text)
  for a in ob_json["asks"]:
    if counts% 2 == 0:
      ask_prices.append(a)
    else:
      ask_sizes.append(a)
    counts += 1
print(ask_prices)
print(ask_sizes)

['0.000625', '0.000627', '0.000629', '0.000649', '0.00065', '0.000658', '0.000667', '0.000668', '0.000679', '0.00068', '0.000684', '0.000687', '0.000689', '0.000694', '0.000698', '0.0007', '0.000705', '0.00072', '0.000728', '0.000731']
['3.19', '25.87', '1.84', '0.05', '40', '0.74', '0.08', '0.99', '0.05', '0.07', '0.10', '0.02', '0.10', '0.10', '0.72', '0.02', '0.02', '2.27', '0.06', '0.04']


In [ ]:
# Binance Historical Data
symbol = "BTCUSDT"
interval = "1d"
candles = requests.get(f"https://data.binance.us/api/v3/klines?symbol={symbol}&interval={interval}")
candles_json = []

if candles.status_code == 200:
  candles_json = json.loads(candles.text)
  print(candles_json)

#**2.Structuring Pairs**



*   Step 0: Gather Correct Coins
*   Step 1: Structure Triangular Pairs

You can find the relevant documentation in the following link: https://docs.poloniex.com#introduction



In [ ]:
# Here we import relevant packages
import pandas as pd                 # Pandas library for data organization and manupulation
import numpy as np
import requests
import time
import json
import collections                  # List operations
from itertools import permutations  # Allows us to check all potential triangular arbitrage opportunities for given list of trading pairs

In [ ]:
def get_coin_tickers(url):
  '''
      This function returns the trade activities at the given url
  '''
  req = requests.get(url)
  coin_json = json.loads(req.text)
  return coin_json

def collect_tradeables(coin_json):
  '''
      This function returns the tradeable pairs at the given json
  '''
  coin_list = set()
  # List of all tradeable coins
  for coin in coin_json:
    #print(coin)
    coin_list.add(coin['symbol'])

  coin_list = list(coin_list) # Here we convert to list
  return coin_list


In [ ]:
def structure_triangular_pairs(coin_list):
  '''
      This function returns the list of pairs that form triangule
  '''
  # Declare Variables
  triangular_pairs_list = []
  remove_duplicates_list = []
  pairs_list = coin_list

  # Get Pair A
  for pair_a in coin_list:
    pair_a_split = pair_a.split("_")
    a_base = pair_a_split[0]
    a_quote = pair_a_split[1]

    # Assign A to a box
    a_pair_box = [a_base, a_quote]

    # Get Pair B
    for pair_b in coin_list:
      pair_b_split = pair_b.split("_")
      b_base = pair_b_split[0]
      b_quote = pair_b_split[1]

      # Check Pair B
      if pair_b != pair_a:
        if  b_base in a_pair_box or b_quote in a_pair_box:

          # Get Pair B
          for pair_c in coin_list:
            pair_c_split = pair_c.split("_")
            c_base = pair_c_split[0]
            c_quote = pair_c_split[1]

          # Count the number of matching C itmes
          if pair_c != pair_a and pair_c != pair_b:
            combine_all = [pair_a, pair_b, pair_c]
            pair_box = [a_base, a_quote, b_base, b_quote, c_base, c_quote]

            counts_c_base = 0
            for i in pair_box:
              if i == c_base:
                counts_c_base += 1

            counts_c_quote = 0
            for i in pair_box:
              if i == c_quote:
                counts_c_quote += 1

          # Determining Triangular Match
          if  counts_c_base == 2 and counts_c_quote == 2 and c_base != c_quote:
            combined = pair_a + "," + pair_b + "," + pair_c
            unique_item = ''.join(sorted(combine_all))

            if unique_item not in remove_duplicates_list:
              match_dict = {
                  "a_base": a_base,
                  "b_base": b_base,
                  "c_base": c_base,
                  "a_quote": a_quote,
                  "b_quote": b_quote,
                  "c_quote": c_quote,
                  "pair_a": pair_a,
                  "pair_b": pair_b,
                  "pair_c": pair_c,
              }
            triangular_pairs_list.append(match_dict)
            remove_duplicates_list.append(unique_item)

  return triangular_pairs_list

In [ ]:
# Extract list of coins and prices from Exchange
url = "https://api.poloniex.com/markets/price"

coin_json = get_coin_tickers(url)          # Here we get the json format of trade activities at the given url
coin_list = collect_tradeables(coin_json)  # Here we get the list of coins
# Here we get the list of triangular pairs list
triangular_pairs_list =structure_triangular_pairs(coin_list)
# Here we save it to JSON File
#with open("structured_triangular_pairs.json", "w") as fp:
#  json.dump(triangular_pairs_list, fp)

print(triangular_pairs_list)

None


#**3. Calculating Surface Rates**

##3.1 Get triangular pairs from external surface

In [ ]:
def getTriangularArbitrageList(splitted_pairs):
  '''
    This function identifies all triangular arbitrage permutations by
    checking if 3 pairs form a set with size 3
  '''
  # Here we form all possible triangular arbitrage permutations
  tri_all_permutations = permutations([x for x in range(0, len(splitted_pairs))], 3)
  # List of an actual triangular pairs by index
  tri_arbitrage_list = []
  Nonexistant = True # Flag to indicate a permutation already exists in the triangular arbitrage list

  for permutation in tri_all_permutations:
    # Here order the permutation to make sure we don't add it more than one with different combination
    permutation_ordered = [list(permutation)[0], list(permutation)[1], list(permutation)[2]]
    # Here we form a set and check if it has 3 elements
    assets_permutation = [splitted_pairs[permutation_ordered[0]],
                              splitted_pairs[permutation_ordered[1]],splitted_pairs[permutation_ordered[2]]]
    all_assets = set().union(*assets_permutation)

    if len(all_assets) == 3:
      # Here we make sure the order forms a cycle such that [A/B, B/C, A/C]
      if(splitted_pairs[permutation_ordered[0]][1] == splitted_pairs[permutation_ordered[2]][0]):
        permutation_ordered = [list(permutation)[0], list(permutation)[2], list(permutation)[1]]
      elif(splitted_pairs[permutation_ordered[1]][1] == splitted_pairs[permutation_ordered[2]][0]):
        permutation_ordered = [list(permutation)[1], list(permutation)[2], list(permutation)[0]]
      elif(splitted_pairs[permutation_ordered[1]][1] == splitted_pairs[permutation_ordered[0]][0]):
        permutation_ordered = [list(permutation)[1], list(permutation)[0], list(permutation)[2]]
      elif(splitted_pairs[permutation_ordered[2]][1] == splitted_pairs[permutation_ordered[0]][0]):
        permutation_ordered = [list(permutation)[2], list(permutation)[0], list(permutation)[1]]
      elif(splitted_pairs[permutation_ordered[2]][1] == splitted_pairs[permutation_ordered[1]][0]):
        permutation_ordered = [list(permutation)[2], list(permutation)[1], list(permutation)[0]]
      else:
        continue # Here the order is correct and we leave it as it is

      # Here we check if a permutation already exists in the list
      for permutation in tri_arbitrage_list:
        if collections.Counter(permutation) == collections.Counter(permutation_ordered) :
          Nonexistant = False
        else:
          Nonexistant = True

      if Nonexistant:
        tri_arbitrage_list.append(permutation_ordered)

  return tri_arbitrage_list

In [ ]:
# Top 500 Binance Trading Pairs based on their trading volume
binance_pairs = pd.read_excel('BinanceTradingPairs.xlsx', sheet_name= 'Final Pairs')['Binance Trading Pairs'].tolist()
# Here we split pairs to form a list of triangular arbitrage cycles
splitted_pairs = [s.split("/") for s in binance_pairs]
# We get all triangular arbitrage pairs
tri_arbitrage_list = getTriangularArbitrageList(splitted_pairs)

In [ ]:
# Here we remove / in trading pairs because naming convention in Poloniex
binance_pairs = [s.replace('/', '_') for s in binance_pairs]
# We form a list of triangular arbitrage to check real time and eliminate pairs which do not form a triangular arbitrage opportunity
triangular_pairs = [[binance_pairs[tri[0]], binance_pairs[tri[1]],
                     binance_pairs[tri[2]]] for tri in tri_arbitrage_list ]
# to convert lists to dictionary with splitted pairs to check if any triangular arbitrage opportunity shows later using getTriangularArbitrageList() function
pairs_dict = {binance_pairs[i]: splitted_pairs[i] for i in range(len(binance_pairs))}
# Here we reduce our list pairs to the ones forming an triangular arbitrage opportunity
pairs_refined = list(set().union(*triangular_pairs))
# Here we refine the pairs dictionary to the ones forming an triangular arbitrage opportunity
pairs_dict = {key: value for key, value in pairs_dict.items() if key in pairs_refined}
print("Number of pairs we track is: ", len(pairs_dict))
print(triangular_pairs)

Number of pairs we track is:  244
[['ID_USDT', 'USDT_TRY', 'ID_TRY'], ['BONK_FDUSD', 'FDUSD_USDT', 'BONK_USDT'], ['BONK_FDUSD', 'FDUSD_TRY', 'BONK_TRY'], ['DOT_BTC', 'BTC_USDC', 'DOT_USDC'], ['DOT_USDC', 'USDC_USDT', 'DOT_USDT'], ['DOT_BTC', 'BTC_USDC', 'DOT_USDC'], ['OP_BTC', 'BTC_USDT', 'OP_USDT'], ['OP_BTC', 'BTC_USDC', 'OP_USDC'], ['ACE_BTC', 'BTC_USDT', 'ACE_USDT'], ['ACE_BTC', 'BTC_TRY', 'ACE_TRY'], ['IMX_BTC', 'BTC_USDT', 'IMX_USDT'], ['REEF_USDT', 'USDT_TRY', 'REEF_TRY'], ['LINK_ETH', 'ETH_BTC', 'LINK_BTC'], ['LINK_ETH', 'ETH_FDUSD', 'LINK_FDUSD'], ['LINK_ETH', 'ETH_USDT', 'LINK_USDT'], ['FLOKI_USDT', 'USDT_TRY', 'FLOKI_TRY'], ['MBOX_USDT', 'USDT_TRY', 'MBOX_TRY'], ['WLD_FDUSD', 'FDUSD_USDT', 'WLD_USDT'], ['WLD_BTC', 'BTC_FDUSD', 'WLD_FDUSD'], ['WLD_FDUSD', 'FDUSD_TRY', 'WLD_TRY'], ['BNB_BTC', 'BTC_USDC', 'BNB_USDC'], ['BNB_ETH', 'ETH_USDC', 'BNB_USDC'], ['BNB_USDC', 'USDC_USDT', 'BNB_USDT'], ['BNB_ETH', 'ETH_USDC', 'BNB_USDC'], ['SOL_BNB', 'BNB_USDC', 'SOL_USDC'], ['BNB_BTC', 

In [ ]:
# Here we save it to JSON File
with open("structured_triangular_pairs.json", "w") as fp:
  json.dump(triangular_pairs, fp)

##3.2 Loading Structured Pairs Fast

In [ ]:
# Set Variables
#coin_price_url = "https://poloniex.com/public?command=returnTicker"

# Get Structured Pairs
with open("structured_triangular_pairs.json") as json_file:
  structured_pairs = json.load(json_file)

# Get Latest Surface Prices
#for key in structured_pairs.keys
print(structured_pairs)

[['ID_USDT', 'USDT_TRY', 'ID_TRY'], ['BONK_FDUSD', 'FDUSD_USDT', 'BONK_USDT'], ['BONK_FDUSD', 'FDUSD_TRY', 'BONK_TRY'], ['DOT_BTC', 'BTC_USDC', 'DOT_USDC'], ['DOT_USDC', 'USDC_USDT', 'DOT_USDT'], ['DOT_BTC', 'BTC_USDC', 'DOT_USDC'], ['OP_BTC', 'BTC_USDT', 'OP_USDT'], ['OP_BTC', 'BTC_USDC', 'OP_USDC'], ['ACE_BTC', 'BTC_USDT', 'ACE_USDT'], ['ACE_BTC', 'BTC_TRY', 'ACE_TRY'], ['IMX_BTC', 'BTC_USDT', 'IMX_USDT'], ['REEF_USDT', 'USDT_TRY', 'REEF_TRY'], ['LINK_ETH', 'ETH_BTC', 'LINK_BTC'], ['LINK_ETH', 'ETH_FDUSD', 'LINK_FDUSD'], ['LINK_ETH', 'ETH_USDT', 'LINK_USDT'], ['FLOKI_USDT', 'USDT_TRY', 'FLOKI_TRY'], ['MBOX_USDT', 'USDT_TRY', 'MBOX_TRY'], ['WLD_FDUSD', 'FDUSD_USDT', 'WLD_USDT'], ['WLD_BTC', 'BTC_FDUSD', 'WLD_FDUSD'], ['WLD_FDUSD', 'FDUSD_TRY', 'WLD_TRY'], ['BNB_BTC', 'BTC_USDC', 'BNB_USDC'], ['BNB_ETH', 'ETH_USDC', 'BNB_USDC'], ['BNB_USDC', 'USDC_USDT', 'BNB_USDT'], ['BNB_ETH', 'ETH_USDC', 'BNB_USDC'], ['SOL_BNB', 'BNB_USDC', 'SOL_USDC'], ['BNB_BTC', 'BTC_USDC', 'BNB_USDC'], ['SOL_BNB

In [61]:
def getTriangularArbitrageList(pair):


  # Structure Orderbook
  ask_prices = []
  ask_sizes = []

  bid_prices = []
  bid_sizes = []

  counts = 0

  ticker = pair
  limit = 20
  ob = requests.get(f"https://api.poloniex.com/markets/{ticker}/orderBook?limit={limit}")
  if ob.status_code == 200:
    ob_json = json.loads(ob.text)
    for a in ob_json["asks"]:
      if counts% 2 == 0:
        ask_prices.append(a)
      else:
        ask_sizes.append(a)
      counts += 1

    for a in ob_json["bids"]:
      if counts% 2 == 0:
        bid_prices.append(a)
      else:
        bid_sizes.append(a)
      counts += 1

    return {"time": pd.to_datetime(ob_json["time"], utc=True, unit='ms').strftime('%Y-%m-%d %H:%M:%S.%f')[:-3],  #Here we convert it to string type to save it as JSON file
            "lowestAsk":  ask_prices[np.argmin(ask_prices)],
            "lowestAskSize":  ask_sizes[np.argmin(ask_prices)],
            "highestBid":  bid_prices[np.argmax(bid_prices)],
            "highestBidSize":  bid_sizes[np.argmax(bid_prices)]
            }


In [62]:
# Here we add ask and bid prices to an arbitrage dictionary
arbitrageDict = {}

for triangle in structured_pairs:
  triangleDict = {}
  triangleName = triangle[0] + "-" +  triangle[1] + "-" +  triangle[2]
  for pair in triangle:
    # In case of missing data we pass it
    try:
      pair_data = getTriangularArbitrageList(pair)
    except:
      pass
    triangleDict.update({pair: pair_data})
  print(triangleDict)
  # Here we add the data to arbitrage dictionary
  arbitrageDict.update({triangleName: triangleDict})


{'ID_USDT': {'time': '2024-03-08 09:32:07.691', 'lowestAsk': '0.7604', 'lowestAskSize': '822.2259', 'highestBid': '0.7071', 'highestBidSize': '41.91'}, 'USDT_TRY': None, 'ID_TRY': None}
{'BONK_FDUSD': None, 'FDUSD_USDT': None, 'BONK_USDT': {'time': '2024-03-08 09:32:07.871', 'lowestAsk': '0.000031099', 'lowestAskSize': '22530206', 'highestBid': '0.00002995', 'highestBidSize': '333890'}}
{'BONK_FDUSD': None, 'FDUSD_TRY': None, 'BONK_TRY': None}
{'DOT_BTC': {'time': '2024-03-08 09:32:10.496', 'lowestAsk': '0.0001542', 'lowestAskSize': '9.9', 'highestBid': '0.0001537', 'highestBidSize': '0.1'}, 'BTC_USDC': {'time': '2024-03-08 09:32:12.058', 'lowestAsk': '67583.19', 'lowestAskSize': '0.000053', 'highestBid': '67443.61', 'highestBidSize': '0.008041'}, 'DOT_USDC': None}
{'DOT_USDC': None, 'USDC_USDT': None, 'DOT_USDT': {'time': '2024-03-08 09:32:14.104', 'lowestAsk': '10.362', 'lowestAskSize': '521.68176', 'highestBid': '10.343', 'highestBidSize': '213.48800'}}
{'DOT_BTC': {'time': '2024-03

In [63]:
# Here we save it to JSON File
with open("triangular_pairs_bid_ask_prices.json", "w") as fp:
  json.dump(arbitrageDict, fp)

In [64]:
# Get Triangular Pairs Price Data
with open("triangular_pairs_bid_ask_prices.json") as json_file:
  arbitrageDict = json.load(json_file)

# Get Latest Surface Prices
print(arbitrageDict)

{'ID_USDT-USDT_TRY-ID_TRY': {'ID_USDT': {'time': '2024-03-08 09:50:32.330', 'lowestAsk': '0.7671', 'lowestAskSize': '822.2259', 'highestBid': '0.7071', 'highestBidSize': '41.91'}, 'USDT_TRY': None, 'ID_TRY': None}, 'BONK_FDUSD-FDUSD_USDT-BONK_USDT': {'BONK_FDUSD': None, 'FDUSD_USDT': None, 'BONK_USDT': {'time': '2024-03-08 09:57:41.891', 'lowestAsk': '0.000031348', 'lowestAskSize': '64364', 'highestBid': '0.000030231', 'highestBidSize': '496179'}}, 'BONK_FDUSD-FDUSD_TRY-BONK_TRY': {'BONK_FDUSD': None, 'FDUSD_TRY': None, 'BONK_TRY': None}, 'DOT_BTC-BTC_USDC-DOT_USDC': {'DOT_BTC': {'time': '2024-03-08 09:44:18.500', 'lowestAsk': '0.0001544', 'lowestAskSize': '133.2', 'highestBid': '0.0001535', 'highestBidSize': '354.6'}, 'BTC_USDC': {'time': '2024-03-08 09:44:25.079', 'lowestAsk': '67788.02', 'lowestAskSize': '0.000421', 'highestBid': '67236.57', 'highestBidSize': '0.06741'}, 'DOT_USDC': None}, 'DOT_USDC-USDC_USDT-DOT_USDT': {'DOT_USDC': None, 'USDC_USDT': None, 'DOT_USDT': {'time': '202

##3.3 Surface Rate Calculation

In [82]:
# Calculate Surface Rate Arbitrage Opportunity
def calc_triangular_arb_surface_rate(structured_pair, arbitrageDict):

  # Set Variables
  starting_amount = 1
  min_surface_rate = 0
  surface_dict = {}
  contract_2 = ""
  contract_3 = ""
  direction_trade_1 = ""
  direction_trade_2 = ""
  direction_trade_3 = ""
  acquired_coin_t2 = 0
  acquired_coin_t3 = 0
  calculated = 0

  # Extract Pair Variables
  a_base = structured_pair[0].split('_')[0]
  a_quote = structured_pair[0].split('_')[1]
  b_base = structured_pair[1].split('_')[0]
  b_quote = structured_pair[1].split('_')[1]
  c_base = structured_pair[2].split('_')[0]
  c_quote = structured_pair[2].split('_')[1]

  pair_a = structured_pair[0]
  pair_b = structured_pair[1]
  pair_c = structured_pair[2]

  # Extract Price Information
  triangleName = structured_pair[0] + "-" +  structured_pair[1] + "-" +  structured_pair[2]

  a_ask = float(arbitrageDict[triangleName][pair_a]['lowestAsk'])
  a_bid = float(arbitrageDict[triangleName][pair_a]['highestBid'])
  b_ask = float(arbitrageDict[triangleName][pair_b]['lowestAsk'])
  b_bid = float(arbitrageDict[triangleName][pair_b]['highestBid'])
  c_ask = float(arbitrageDict[triangleName][pair_c]['lowestAsk'])
  c_bid = float(arbitrageDict[triangleName][pair_c]['highestBid'])

  # Set directions and loop through
  direction_list = ["forward", "reverse"]

  for direction in direction_list:
    # Set additional variables for swap information
    swap_1 = 0
    swap_2 = 0
    swap_3 = 0
    swap_1_rate = 0
    swap_2_rate = 0
    swap_3_rate = 0

    """
        If we are swapping the coin on the left (Base) to the right (Quote) then multiply 1/Ask
        If we are swapping the coin on the right (Quote) to the left (Base) then multily Bid
    """
    # Assume starting with a_base and swapping for a_quote
    if direction == "forward":
      swap_1 = a_base
      swap_2 = a_quote
      swap_1_rate = 1./a_ask
      direction_trade_1 = "base_to_quote"

    if direction == "reverse":
      swap_1 = a_quote
      swap_2 = a_base
      swap_1_rate = a_bid
      direction_trade_1 = "quote_to_bid"

    # Place First Trade
    contract_1 = pair_a
    acquired_coin_t1 = starting_amount * swap_1_rate
    print(direction, pair_a, starting_amount, acquired_coin_t1)

    """
      FORWARD
      Check if a_quote (acquired_coin) matches b_quote
    """
    if direction == "forward":
      if a_quote == b_quote and calculated == 0:
        swap_2_rate = b_bid
        acquired_coin_t2 = acquired_coin_t1 * swap_2_rate
        direction_trade_2 = "quote_to_base"
        contract = pair_b

        # If b_base (acquired coin) matches c_base
        if b_base == c_base:
          swap_3 = c_base
          swap_3_rate = 1./ c_ask
          direction_trade_3 = "base_to_quote"
          contract_3 = pair_c

        # If b_base (acquired coin) matches c_quote
        if b_base == c_quote:
          swap_3 = c_quote
          swap_3_rate = c_bid
          direction_trade_3 = "quote_to_base"
          contract_3 = pair_c

        acquired_coin_t3 = acquired_coin_t2 * swap_3_rate
        calculated = 1

  print(a_ask, a_bid, pair_b)
  print(arbitrageDict[triangleName])

In [83]:
calc_triangular_arb_surface_rate(structured_pairs[36], arbitrageDict)

forward ADA_BTC 1 91324.200913242
reverse ADA_BTC 1 1.09e-05
1.095e-05 1.09e-05 BTC_USDT
{'ADA_BTC': {'time': '2024-03-08 09:45:18.249', 'lowestAsk': '0.00001095', 'lowestAskSize': '618', 'highestBid': '0.0000109', 'highestBidSize': '2328'}, 'BTC_USDT': {'time': '2024-03-08 09:45:43.082', 'lowestAsk': '67197.78', 'lowestAskSize': '0.098454', 'highestBid': '67154.21', 'highestBidSize': '0.14'}, 'ADA_USDT': {'time': '2024-03-08 09:45:43.484', 'lowestAsk': '0.73595', 'lowestAskSize': '204.6529', 'highestBid': '0.73442', 'highestBidSize': '2227.5541'}}


In [84]:
for triangle in structured_pairs:
  try:
    calc_triangular_arb_surface_rate(triangle, arbitrageDict)
  except:
    pass

forward FTM_BTC 1 81833.06055646481
reverse FTM_BTC 1 1.198e-05
1.222e-05 1.198e-05 BTC_USDT
{'FTM_BTC': {'time': '2024-03-08 09:44:30.758', 'lowestAsk': '0.00001222', 'lowestAskSize': '410', 'highestBid': '0.00001198', 'highestBidSize': '14'}, 'BTC_USDT': {'time': '2024-03-08 09:46:12.043', 'lowestAsk': '67187.88', 'lowestAskSize': '0.098542', 'highestBid': '67162.22', 'highestBidSize': '0.050903'}, 'FTM_USDT': {'time': '2024-03-08 09:45:56.625', 'lowestAsk': '0.9001', 'lowestAskSize': '783.2081', 'highestBid': '0.8024', 'highestBidSize': '1.8694'}}
forward GAS_BTC 1 9216.589861751152
reverse GAS_BTC 1 0.0001055
0.0001085 0.0001055 BTC_USDT
{'GAS_BTC': {'time': '2024-03-08 09:30:06.908', 'lowestAsk': '0.0001085', 'lowestAskSize': '0.5', 'highestBid': '0.0001055', 'highestBidSize': '0.1'}, 'BTC_USDT': {'time': '2024-03-08 09:46:11.091', 'lowestAsk': '67187.7', 'lowestAskSize': '0.14', 'highestBid': '67162.2', 'highestBidSize': '0.050904'}, 'GAS_USDT': {'time': '2024-03-08 09:21:00.947'

#**4.Calculating Depth**

In [ ]:
import asyncio

async def stopwatch():
  count = 0
  while count < 4:
      await asyncio.sleep(1)
      count += 1
      print(count)

def callb(task):
  print('task is done', task)

async def main():
  task = asyncio.create_task(stopwatch())
  task.set_name('My Task')
  task.add_done_callback(callb)
  print(task.get_name())
  print(task.get_coro())
  print(task.get_name())
  await task

await main()

My Task
<coroutine object stopwatch at 0x7ea2787a4660>
My Task
1
2
3
4
task is done <Task finished name='My Task' coro=<stopwatch() done, defined at <ipython-input-150-b87e2fe8d0a6>:3> result=None>


##3.2 Interacting with the Event Loop:

In [ ]:
import asyncio

async def stopwatch():
  count = 0
  while count < 4:
      await asyncio.sleep(1)
      count += 1
      print(count)

def callb(task):
  print('task is done', task)

async def main():
  print(asyncio.get_running_loop())
  task = asyncio.create_task(stopwatch())
  task.add_done_callback(callb)

  await task

await main()

<_UnixSelectorEventLoop running=True closed=False debug=False>
1
2
3
4
task is done <Task finished name='Task-69' coro=<stopwatch() done, defined at <ipython-input-151-f16058d61056>:3> result=None>


##3.3 The concept of the future:

In [ ]:
import asyncio

async def stopwatch():
  count = 0
  while count < 4:
      await asyncio.sleep(1)
      count += 1
      print(count)

def callb(task):
  print('task is done', task)

async def main():
  task = asyncio.create_task(stopwatch())
  task.add_done_callback(callb)
  print(asyncio.isfuture(task))

  await task

await main()

True
1
2
3
4
task is done <Task finished name='Task-71' coro=<stopwatch() done, defined at <ipython-input-152-fd9b0fb67711>:3> result=None>
